In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
df_train = TabularDataset("/DATA/jimson/Rishi/Priyadrasta/papercode/third floor/fold 3/Tabular_train3.csv")
df_test = TabularDataset("/DATA/jimson/Rishi/Priyadrasta/papercode/third floor/fold 3/Tabular_test3.csv")


In [ ]:
df_train.shape,df_test.shape

In [ ]:
print(df_train['label'].value_counts(),df_test['label'].value_counts(),sep="\n")

In [ ]:
test_data=df_test.drop(['label'],axis=1)
test_data.head()

In [ ]:
# predictor= TabularPredictor(label ='label',problem_type='multiclass',eval_metric='accuracy').fit(train_data = df_train,time_limit=3600,verbosity = 2,presets='best_quality'
# #  ,hyperparameters={'NN':{}, 'CAT':{}, 'RF':{}, 'XT':{}, 'KNN':{}}
# )
predictor=TabularPredictor.load("/DATA/jimson/Rishi/Priyadrasta/AutoMM/AutogluonModels/ag-20230118_044740")
# predictor=TabularPredictor.load('/DATA/jimson/Rishi/Priyadrasta/papercode/third floor/fold 2/AutogluonModels/ag-20230104_134229')

In [ ]:
predictor.leaderboard(df_test,extra_metrics=['accuracy'], silent=True)

In [ ]:
predictor.feature_importance(data=df_train)

In [ ]:
predictor.evaluate(df_test)

In [ ]:
test_data

In [ ]:
prediction = predictor.predict(test_data)
prediction =pd.DataFrame(prediction,columns=['label'])
prediction.value_counts()


In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix= confusion_matrix(df_test['label'].to_numpy(),prediction.to_numpy())
conf_matrix

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
 
fig, ax = plot_confusion_matrix(conf_mat=conf_matrix, figsize=(4, 4), cmap=plt.cm.Blues)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix) 
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

In [ ]:
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
print("class-wise  Accuracy :",ACC)
print("class-wise  Precision :",PPV)
print("class-wise  Sensitivity :",TPR)
print("class-wise  Specificity :",TNR)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_test['label'].to_numpy(), prediction.to_numpy()))

In [ ]:
probas = predictor.predict_proba(test_data)
probas.head(5)


In [ ]:
probas_arr=probas.values
print(probas_arr.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
Combined_test=df_test.drop(['label'],axis=1)
Combined_label=df_test['label']

macro_roc_auc_ovr = roc_auc_score(Combined_label, probas, multi_class="ovr", average="macro")
# macro_roc_auc_ovr = roc_auc_score(Combined_prediction, df_test['prediction'], multi_class="ovr", average="macro")

weighted_roc_auc_ovr = roc_auc_score(
    Combined_label, probas, multi_class="ovr", average="weighted")


print(
    "One-vs-Rest ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovr, weighted_roc_auc_ovr))

In [ ]:
macro_roc_auc_ovo = roc_auc_score(Combined_label, probas, multi_class="ovo", average="macro")
weighted_roc_auc_ovo = roc_auc_score(
    Combined_label, probas, multi_class="ovo", average="weighted"
)

print(
    "One-vs-One ROC AUC scores:\n{:.6f} (macro),\n{:.6f} "
    "(weighted by prevalence)".format(macro_roc_auc_ovo, weighted_roc_auc_ovo)
)

In [ ]:
arr_comb_test=df_test.values
# arr_comb_test


In [ ]:
combined_test=arr_comb_test[:, 0:4]
combined_prediction=arr_comb_test[:, 4]

In [ ]:
combined_label = label_binarize(combined_prediction, classes=[1, 2, 3, 4])
combined_label.shape

In [ ]:
combined_label = label_binarize(combined_label, classes=[1, 2, 3,4])
n_classes = combined_label.shape[1]

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(combined_label[:, i], probas_arr[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(combined_label.ravel(), probas_arr.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
lw = 2
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label="micro-average ROC curve (area = {0:0.2f})".format(roc_auc["micro"]),
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.2f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

colors = cycle(["aqua", "darkorange", "cornflowerblue","red"])
for i, color in zip(range(n_classes), colors):
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i+1, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Auto-Gluon")
plt.legend(loc="lower right")
plt.show()

In [ ]:
df_train

In [ ]:
# df_binary_train=df_train.copy()
# df_binary_train['label_a']=df_binary_train['label_1']+df_binary_train['label_2']
# df_binary_train['label_b']=df_binary_train['label_3']+df_binary_train['label_4']
# df_binary_train.drop(columns = ['label_1',	'label_2'	,'label_3'	,'label_4'],inplace=True)
# df_binary_train=df_binary_train.reindex(columns=['label_a','label_b','label'])
# df_binary_train['label'] = df_binary_train['label'].apply(lambda x:int (x>2))
# df_binary_train.rename(columns={"label_a":"label_0","label_b":"label_1"},inplace=True)
# df_binary_train

In [ ]:

df_binary_train = TabularDataset("/DATA/jimson/Rishi/Priyadrasta/papercode/third floor/fold 3/Tabular_train_binary3.csv")
df_binary_test = TabularDataset("/DATA/jimson/Rishi/Priyadrasta/papercode/third floor/fold 3/Tabular_test_binary3.csv")

In [ ]:
# df_binary_train,df_binary_test=train_test_split(df_binary,test_size=0.29,random_state=1)
df_binary_train.shape,df_binary_test.shape

In [ ]:
df_binary_test['label'].value_counts()

In [ ]:
test_data=df_binary_test.drop(['label'],axis=1)
test_data.head()

In [ ]:
binary_predictor= TabularPredictor.load("/DATA/jimson/Rishi/Priyadrasta/papercode/third floor/fold 2/AutogluonModels/ag-20230104_065039")
# TabularPredictor(label ='label',problem_type='binary',eval_metric='roc_auc').fit(train_data = df_binary_train, verbosity = 2,presets='best_quality'
#  ,hyperparameters={'NN':{}, 'CAT':{}, 'RF':{}, 'XT':{}, 'KNN':{}}
# )

In [ ]:
binary_predictor.leaderboard(df_binary_test,extra_metrics=['accuracy'], silent=True)

In [ ]:
binary_predictor.feature_importance(data=df_binary_train)

In [ ]:
binary_predictor.evaluate(df_binary_test)

In [ ]:
prediction = binary_predictor.predict(test_data)
prediction =pd.DataFrame(prediction,columns=['label'])
prediction.value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix= confusion_matrix(df_binary_test['label'].to_numpy(),prediction.to_numpy())
conf_matrix

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
 
fig, ax = plot_confusion_matrix(conf_mat=conf_matrix, figsize=(4, 4), cmap=plt.cm.Reds)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix) 
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

In [ ]:
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
print("class-wise  Accuracy :",ACC)
print("class-wise  Precision :",PPV)
print("class-wise  Sensitivity :",TPR)
print("class-wise  Specificity :",TNR)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_binary_test['label'].to_numpy(), prediction.to_numpy()))